# 02 · BLS Baseline
以 Lightkurve 下載光曲線 → 去趨勢 → BLS → 摺疊與視覺化

In [ ]:
# 環境設定與依賴安裝（Colab）
import sys, subprocess, pkgutil

def pipi(*pkgs):
    """安裝套件的輔助函式"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])

# 安裝必要套件（避免 numpy 2.0 相容性問題）
print("🚀 正在安裝依賴套件...")
try:
    import numpy as np
    import lightkurve as lk
    print("✅ 基礎套件已安裝")
except Exception:
    pipi("numpy<2", "lightkurve", "astroquery", "scikit-learn", "matplotlib", "wotan", "transitleastsquares")
    print("✅ 依賴套件安裝完成")

# 檢查 GPU 資訊
import torch if 'torch' in [m.name for m in pkgutil.iter_modules()] else None
if torch is not None and torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🖥️ GPU 型號: {gpu_name}")
    print(f"   記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    
    # 如果是 NVIDIA L4，提供 BF16 優化建議
    if "L4" in gpu_name:
        print("💡 偵測到 NVIDIA L4 GPU - 支援高效能 BF16 運算")
        print("   建議在訓練時使用 torch.autocast('cuda', dtype=torch.bfloat16)")
else:
    try:
        # 使用 nvidia-smi 檢查 GPU
        result = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], 
                              capture_output=True, text=True, check=False)
        if result.returncode == 0:
            gpu_name = result.stdout.strip()
            print(f"🖥️ GPU 型號: {gpu_name}")
            if "L4" in gpu_name:
                print("💡 偵測到 NVIDIA L4 GPU - 支援高效能 BF16 運算")
    except:
        print("⚠️ 未偵測到 GPU，將使用 CPU 運算")

print("\n環境設定完成！")

In [ ]:
import lightkurve as lk, numpy as np, matplotlib.pyplot as plt
target = "TIC 25155310"  # 可更換
sr = lk.search_lightcurve(target, mission="TESS", author="SPOC")
lc = sr.download().remove_nans()
flat = lc.flatten(window_length=401)
bls = flat.to_periodogram(method="bls", minimum_period=0.5, maximum_period=20)
period = bls.period_at_max_power
t0     = bls.transit_time_at_max_power
dur    = bls.duration_at_max_power
snr    = bls.max_power
period, t0, dur, snr

In [ ]:
ax = bls.plot(); ax.set_title("BLS Power Spectrum");

In [ ]:
# 摺疊光曲線
folded = flat.fold(period, t0)
ax = folded.plot()
ax.set_title(f"Folded Light Curve (P={period.value:.3f} d)");